<a href="https://colab.research.google.com/github/avilash01/Rag/blob/main/Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
corpus_of_documents=[
  "Take a leiasurely walk in the park and enjoy the fresh air.",
  "Visit a local musuem and discover something new.",
  "Attend a live music concert and feel the rhythm.",
  "Go for a hike and admire the natural scenery.",
  "Have a picnic with friends and share some laughs.",
  "Explore a new cuisine by dining at ethnic restaurant.",
  "I am a fresher.",
  "I am interested in rag."
]

In [36]:
corpus_of_documents

['Take a leiasurely walk in the park and enjoy the fresh air.',
 'Visit a local musuem and discover something new.',
 'Attend a live music concert and feel the rhythm.',
 'Go for a hike and admire the natural scenery.',
 'Have a picnic with friends and share some laughs.',
 'Explore a new cuisine by dining at ethnic restaurant.',
 'I am a fresher.',
 'I am interested in rag.']

In [37]:
query="I am an indian and I love in india"

In [38]:
doc="india is a country for the indians and for everyone"

In [39]:
from collections import Counter
import math

In [40]:
query_tokens=query.lower().split(" ")

In [41]:
doc="India is a country for the Indian"
doc_tokens=doc.lower().split(" ")

In [42]:
query_counter=Counter(query_tokens)

In [43]:
doc_counter=Counter(doc_tokens)

In [44]:
doc_counter

Counter({'india': 1,
         'is': 1,
         'a': 1,
         'country': 1,
         'for': 1,
         'the': 1,
         'indian': 1})

In [45]:
list=[]
for tken in query_counter.keys():
        list.append(query_counter[tken])

In [46]:
list

[2, 1, 1, 1, 1, 1, 1, 1]

In [47]:
mylist=[]
common_tokens = query_counter.keys() & doc_counter.keys()
for tken in common_tokens:
        mylist.append(query_counter[tken]*doc_counter[tken])

In [48]:
mylist

[1, 1]

In [49]:
dot_prod=sum(mylist)

In [50]:
query_magnitude=math.sqrt(sum(query_counter[tken]** 2 for tken in query_counter))

In [51]:
query_magnitude

3.3166247903554

In [52]:
doc_magnitude=math.sqrt(sum(doc_counter[tken]** 2 for tken in doc_counter))

In [53]:
doc_magnitude

2.6457513110645907

In [54]:
similarity=dot_prod/(query_magnitude*doc_magnitude)

In [55]:
similarity

0.2279211529192759

In [56]:
user_query="is yoga good for health"
document="yoga is very good health"

In [57]:
def cosine_simailarity(query,document):
  query_tokens=query.lower().split(" ")
  doc_tokens=document.lower().split(" ")
  query_counter=Counter(query_tokens)
  doc_counter=Counter(doc_tokens)

  dot_prod=sum(query_counter[tken]*doc_counter[tken] for tken in query_counter.keys() & doc_counter.keys())
  query_magnitude=math.sqrt(sum(query_counter[tken]** 2 for tken in query_counter))
  doc_magnitude=math.sqrt(sum(doc_counter[tken]** 2 for tken in doc_counter))
  similarity=dot_prod/(query_magnitude*doc_magnitude)
  return similarity

In [58]:
cosine_simailarity(user_query,document)

0.7999999999999998

In [59]:
def return_response(query,corpus):
  similarities=[]
  for doc in corpus:
    similarities.append(cosine_simailarity(query,doc))
  return corpus_of_documents[similarities.index(max(similarities))]

In [60]:
query="I like fresh air"

In [61]:
return_response(query,corpus_of_documents)

'I am a fresher.'

In [62]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [63]:

import subprocess
import time

process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("Ollama server started.")

Ollama server started.


In [71]:
import requests
import json
import subprocess
import time

full_response=[]

prompt="""
You are a bot that makes recommedations for activities.You answer in very short sentences and do not.This is the recommded
activity:{relevant_document}
the user input is {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""

url = "http://localhost:11434/api/generate"

subprocess.run(['killall', 'ollama'], capture_output=True, text=True)
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(10)
user_input = query
relevant_document = return_response(query,corpus_of_documents)

data={
  "model": "llama2",
  "prompt": prompt.format(user_input=user_input,relevant_document=relevant_document)
}

headers={"Content-Type":'application/json'}

response=requests.post(url,json=data,headers=headers,stream=True)

try:
  full_response = []
  for line in response.iter_lines():
    if line:
      decoded_line=json.loads(line.decode('utf-8'))
      full_response.append(decoded_line['response'])

finally:
  response.close()
  print(''.join(full_response))

Ollama server re-started.
 Great! Based on your input "I like fresh air," I recommend a visit to a nearby park or beach for some outdoor fun. The crisp air and scenic views will definitely help you relax and enjoy yourself!
